In [10]:
import numpy as np
import pandas as pd
import torch
import jieba as jb
import math
import time
import pickle
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from torch import optim
import torch.nn.functional as F
from gensim.models import Word2Vec
import random
import pdb
embedding_size = 256
embed_type = "word" # "char"
use_cuda = torch.cuda.is_available()
use_cuda

True

In [2]:
chats = pickle.load(open("chat.pkl", "rb"))
flat_list = [list(item) for sublist in chats for item in sublist]

In [3]:
wmodel = Word2Vec(flat_list, size=512, sg=1, workers=10)
wmodel.save("cmodel512.w2v")

In [4]:
word2ix = {"<BOS>": 0, "<EOS>": 1, "<PAD>": 2, "<UNK>": 3}
ix2word = {0: "<BOS>", 1: "<EOS>", 2: "<PAD>", 3: "<UNK>"}
next_ix = 4
for i, sent in enumerate(flat_list):
    for j, word in enumerate(sent):
        if word in wmodel.wv.vocab and word not in word2ix:
            word2ix[word] = next_ix
            ix2word[next_ix] = word
            next_ix += 1
        elif word not in wmodel.wv.vocab:
            flat_list[i][j] = "<UNK>"
weights = []
for ix, word in sorted(ix2word.items(), key=lambda x: x[0]):
    if word in wmodel.wv.vocab:
        weights.append(wmodel.wv[word])
    else:
        weights.append(list(np.random.normal(size=embedding_size)))
weights = torch.tensor(weights)

In [5]:
s1 = []
s2 = []
for i in range(len(flat_list)):
    if i % 2 == 0:
        s1.append(flat_list[i])
    else:
        s2.append(flat_list[i])

In [6]:
pickle.dump(word2ix, open("models/word2ix", "wb"))
pickle.dump(ix2word, open("models/ix2word", "wb"))
pickle.dump(s1, open("models/s1", "wb"))
pickle.dump(s2, open("models/s2", "wb"))
pickle.dump(weights, open("models/weights512", "wb"))

In [13]:
word2ix = pickle.load(open("models/word2ix", "rb"))
ix2word = pickle.load(open("models/ix2word", "rb"))
s1 = pickle.load(open("models/s1", "rb"))
s2 = pickle.load(open("models/s2", "rb"))
w2v = Word2Vec.load("models/cmodel.w2v")
weights = pickle.load(open("models/weights", "rb"))
vocab_size = len(word2ix)
MAXLEN = 20
print(vocab_size)

4475


In [3]:
class ChatDataset(Dataset):
    def __init__(self, qs, anss, word2ix):
        self.qs = qs
        self.anss = anss
        self.pairs_ix = self.construct_data(word2ix)
    def construct_data(self, word2ix):
        pairs_ix = []
        for q, ans in zip(self.qs, self.anss):
            q = q[:MAXLEN]
            ans = ans[:MAXLEN]
            q_ix = [word2ix["<BOS>"]] + [word2ix[word] for word in q] + [word2ix["<EOS>"]]
            pad_num = MAXLEN + 2 - len(q_ix)
            q_ix += [word2ix["<PAD>"]] * pad_num
            ans_ix = [word2ix["<BOS>"]] + [word2ix[word] for word in ans] + [word2ix["<EOS>"]]
            pad_num = MAXLEN + 2 - len(ans_ix)
            ans_ix += [word2ix["<PAD>"]] * pad_num
            q_ix = self.add_pos(q_ix, MAXLEN+2)
            ans_ix = self.add_pos(ans_ix, MAXLEN+2)
            pairs_ix.append([q_ix, ans_ix])
        return np.array(pairs_ix)
    
    def add_pos(self, seq, l):
        seq = np.array(seq).reshape(l, 1)
        pos = np.arange(1, l+1).reshape(l, 1)
        return np.concatenate((seq, pos), 1)
        
    def __len__(self):
        return len(self.qs)
    
    def __getitem__(self, ix):
        pair = [self.qs[ix], self.anss[ix]]
        q = torch.LongTensor(self.pairs_ix[ix][0])
        ans = torch.LongTensor(self.pairs_ix[ix][1])
        return {"pair": pair, "q": q, "ans": ans}

In [4]:
indices = np.arange(len(s1))
np.random.seed(9987)
np.random.shuffle(indices)
qs, anss = np.array(s1)[indices], np.array(s2)[indices]
val_num = int(0.02 * len(qs))
train_qs, train_anss = qs[:-val_num], anss[:-val_num]
val_qs, val_anss = qs[-val_num:], anss[-val_num:]
qa_set = ChatDataset(train_qs, train_anss, word2ix)
vqa_set = ChatDataset(val_qs, val_anss, word2ix)
len(qa_set), len(vqa_set)

(258601, 5277)

In [5]:
def position_encoding_init(n_position, d_pos_vec):
    ''' Init the sinusoid position encoding table '''
    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / d_pos_vec) for j in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    return torch.from_numpy(position_enc).type(torch.FloatTensor)
def get_attn_padding_mask(seq_q, seq_k):
    ''' Indicate the padding-related part to mask '''
    assert seq_q.dim() == 2 and seq_k.dim() == 2
    mb_size, len_q = seq_q.size()
    mb_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(2).unsqueeze(1)   # bx1xsk
    pad_attn_mask = pad_attn_mask.expand(mb_size, len_q, len_k) # bxsqxsk
    return pad_attn_mask

def get_attn_subsequent_mask(seq):
    ''' Get an attention mask to avoid using the subsequent info.'''
    assert seq.dim() == 2
    attn_shape = (seq.size(0), seq.size(1), seq.size(1))
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    subsequent_mask = torch.from_numpy(subsequent_mask)
    if seq.is_cuda:
        subsequent_mask = subsequent_mask.cuda()
    return subsequent_mask

class LayerNorm(nn.Module):
    def __init__(self, hidden_size):
        super(LayerNorm, self).__init__()
        self.a = nn.Parameter(torch.ones(hidden_size))
        self.b = nn.Parameter(torch.zeros(hidden_size))
        self.eps = 1e-6
    def forward(self, x):
        mu = x.mean(-1, keepdim=True)
        sigma = x.std(-1, keepdim=True)
        output = self.a * (x - mu) / (sigma) + self.b
        return output

class ScaledDotProductAttn(nn.Module):
    def __init__(self, hidden_size):
        super(ScaledDotProductAttn, self).__init__()
        self.scaler = hidden_size ** 0.5
    def forward(self, q, k, v, attn_mask=None):
        # batch, qlen, d_head * batch, d_head, klen
        attn_weight = torch.bmm(q, k.transpose(1,2)) / self.scaler
        if attn_mask is not None:
            attn_weight.data.masked_fill_(attn_mask, -float('inf'))
        # batch, qlen, klen
        attn_weight = F.softmax(attn_weight, dim=2)
        attn_weight = torch.bmm(attn_weight, v)
        return attn_weight

class MultiHeadAttn(nn.Module):
    def __init__(self, hidden_size, n_head=8):
        super(MultiHeadAttn, self).__init__()
        self.hidden_size = hidden_size
        self.n_head = n_head
        self.d_head = int(hidden_size / n_head)
        self.linears = nn.ModuleList([
            nn.Linear(hidden_size, hidden_size) for _ in range(4)
        ])
        [init.xavier_normal_(ll.weight) for ll in self.linears]
        self.attn = ScaledDotProductAttn(hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = LayerNorm(hidden_size)
    def forward(self, q, k, v, attn_mask=None):
        batch_size, len_q, _ = q.size()
        len_k = k.size(1)
        # batch, qlen, emb_size
        residual = q
        qs, ks, vs = \
            [l(x).view(batch_size, -1, self.n_head, self.d_head).transpose(1, 2) \
             for l, x in zip(self.linears, (q, k, v))]
        qs = qs.contiguous().view(-1, len_q, self.d_head)
        ks = ks.contiguous().view(-1, len_k, self.d_head)
        vs = vs.contiguous().view(-1, len_k, self.d_head)        
        # SDP attn: batch*n_head, seqlen, d_head
        outputs = self.attn(qs, ks, vs, attn_mask.repeat(self.n_head,1,1))
        # concat: batch, seqlen, hidden_size
        outputs = outputs.view(-1, self.n_head, len_q, self.d_head).transpose(1,2)\
                        .contiguous().view(-1, len_q, self.n_head*self.d_head)
        outputs = self.linears[-1](outputs)
        return self.layer_norm(residual + self.dropout(outputs))

class PositionWiseDense(nn.Module):
    def __init__(self, hidden_size):
        super(PositionWiseDense, self).__init__()
        self.fc1 = nn.Linear(hidden_size, hidden_size*4)
        self.fc2 = nn.Linear(hidden_size*4, hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = LayerNorm(hidden_size)
    def forward(self, x):
        residual = x
        output = self.fc2(F.relu(self.fc1(x)))
        return self.layer_norm(residual + self.dropout(output))

class EncodeSubLayer(nn.Module):
    def __init__(self, hidden_size):
        super(EncodeSubLayer, self).__init__()
        self.multi_head_attn = MultiHeadAttn(hidden_size)
        self.pos_wise_fc = PositionWiseDense(hidden_size)
    def forward(self, enc_input, slf_attn_mask=None):
        output = self.multi_head_attn(enc_input, enc_input, enc_input, slf_attn_mask)
        output = self.pos_wise_fc(output)
        return output

class DecodeSubLayer(nn.Module):
    def __init__(self, hidden_size):
        super(DecodeSubLayer, self).__init__()
        self.multi_head_attn = MultiHeadAttn(hidden_size)
        self.enc_attn = MultiHeadAttn(hidden_size)
        self.pos_wise_fc = PositionWiseDense(hidden_size)
    def forward(self, dec_input, enc_output, slf_attn_mask=None, ende_attn_padding_mask=None):
        dec_output = self.multi_head_attn(dec_input, dec_input, dec_input, slf_attn_mask)
        dec_output = self.enc_attn(dec_output, enc_output, enc_output, ende_attn_padding_mask)
        dec_output = self.pos_wise_fc(dec_output)
        return dec_output

class Encoder(nn.Module):
    def __init__(self, embedding_size, emb_weight, hidden_size):
        super(Encoder, self).__init__()
        self.n_pos = MAXLEN + 3
        self.hidden_size = hidden_size
        self.pos_enc = nn.Embedding(self.n_pos, embedding_size)
        self.pos_enc.weight.data = position_encoding_init(self.n_pos, embedding_size)
        self.seq_embedding = nn.Embedding.from_pretrained(emb_weight, freeze=False)
        self.dropout = nn.Dropout(p=0.1)
        self.layers = nn.ModuleList([
            EncodeSubLayer(hidden_size) for _ in range(6)
        ])

    def forward(self, src):
        seq, pos = src[:,:,0], src[:,:,1]
        slf_attn_padding_mask = get_attn_padding_mask(seq, seq)
        enc_input = self.seq_embedding(seq)
        pos_emb = self.pos_enc(pos)
        enc_input += pos_emb
        enc_output = self.dropout(enc_input)
        for layer in self.layers:
            enc_output = layer(enc_output, slf_attn_mask=slf_attn_padding_mask)
        return enc_output

class Decoder(nn.Module):
    def __init__(self, embedding_size, emb_weight, hidden_size, vocab_size):
        super(Decoder, self).__init__()
        self.n_pos = MAXLEN + 2
        self.hidden_size = hidden_size
        self.pos_enc = nn.Embedding(self.n_pos, embedding_size)
        self.pos_enc.weight.data = position_encoding_init(self.n_pos, embedding_size)
        self.seq_embedding = nn.Embedding.from_pretrained(emb_weight, freeze=False)
        self.layers = nn.ModuleList([
            DecodeSubLayer(hidden_size) for _ in range(6)
        ])
        self.dropout = nn.Dropout(p=0.1)
        self.output_layer = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, target, enc_output, src):
        seq, pos = target[:,:self.n_pos-1,0], target[:,:self.n_pos-1,1]
        
        slf_attn_padding_mask = get_attn_padding_mask(seq, seq)
        slf_attn_sub_mask = get_attn_subsequent_mask(seq)
        slf_attn_mask = torch.gt(slf_attn_padding_mask + slf_attn_sub_mask, 0)
        ende_attn_padding_mask = get_attn_padding_mask(seq, src[:,:,0])
        
        dec_input = self.seq_embedding(seq)
        pos_emb = self.pos_enc(pos)
        dec_input += pos_emb
        dec_output = self.dropout(dec_input)
        for layer in self.layers:
            dec_output = layer(dec_output, enc_output, slf_attn_mask, ende_attn_padding_mask)
        dec_output = self.softmax(self.output_layer(dec_output))
        return dec_output
class Transformer(nn.Module):
    def __init__(self, embedding_size, emb_weight, hidden_size, vocab_size):
        super(Transformer, self).__init__()
        self.encoder = Encoder(embedding_size, emb_weight, hidden_size)
        self.decoder = Decoder(embedding_size, emb_weight, hidden_size, vocab_size)
    
    def forward(self, q_seq, ans_seq):
        enc_output = self.encoder(q_seq)
        dec_output = self.decoder(ans_seq, enc_output, q_seq)
        return dec_output

In [6]:
def validating(vdataloader, encoder, decoder, batch_size, file=None):
    avg_bleu = []
    sf = SmoothingFunction()
    for i, data in enumerate(vdataloader):
        if i == 6:
            avg_bleu = np.mean(avg_bleu)
            return avg_bleu
        q_seq = data['q'].to(device)
        ans_seq = data['ans'].to(device)

        enc_output = encoder(q_seq)
        for i in range(MAXLEN-1):
            if i == 0:
                dec_output = decoder(ans_seq[:,:i+1,:], enc_output, q_seq)
            else:
                dec_input = torch.cat((ans_seq[:,:1,0], dec_output.max(-1)[-1]), 1).unsqueeze(2)
                dec_pos = torch.arange(1, i+2).long().unsqueeze(0).repeat(batch_size, 1).unsqueeze(2).to(device)
                dec_input = torch.cat((dec_input, dec_pos), 2)
                dec_output = decoder(dec_input, enc_output, q_seq)
    #     pdb.set_trace()
        pred = dec_output.max(2)[-1]
        for j in range(batch_size):
            src = [ix2word[wix.item()] for wix in q_seq[j,:,0] if wix.item() != 2]
            trt = [ix2word[wix.item()] for wix in ans_seq[j,:,0] if wix.item() != 2]
            last_words = []
            for wix in pred[j]:
                word = ix2word[wix.item()]
                last_words.append(word)
                if word == "<EOS>":
                    break
                if len(set(last_words[-3:])) == 1 and len(last_words) >= 3:
                    last_words = last_words[:-2]
                    break
            output = last_words
            print(output)
            try:
                avg_bleu.append(sentence_bleu([trt], output, smoothing_function=sf.method4))
            except:
                continue
            if file is not None:
                try:
                    file.write("".join(src) + "\n")
                    file.write("".join(trt) + "\n")
                    file.write("".join(output) + "\n\n")
                except:
                    continue

In [14]:
device = torch.device("cuda" if use_cuda else "cpu")
epochs = 12
vocab_size = len(word2ix)
hidden_size = 256
batch_size = 64
iter_size = len(qa_set) // batch_size
transformer = Transformer(embedding_size, weights, hidden_size, vocab_size).to(device)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "max", factor=0.5, patience=2)
dataloader = DataLoader(qa_set, batch_size=batch_size, shuffle=True)
vdataloader = DataLoader(vqa_set, batch_size=batch_size, shuffle=False)
iter_size

4040

In [15]:
print("Start Training")
transformer.train()
max_bleu = 0
for epoch in range(epochs):
    epoch_loss = 0
    t1 = time.time()
    for i, data in enumerate(dataloader):
        if i % 200 == 0:
            print(i, end=" ")
        loss = 0
        optimizer.zero_grad()
        q_seq = data['q'].to(device)
        ans_seq = data['ans'].to(device)
        
        dec_output = transformer(q_seq, ans_seq)

        loss = F.nll_loss(dec_output.view(-1, vocab_size), ans_seq[:,1:,0].contiguous().view(-1))
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    t2 = time.time()
    transformer.eval()
    bleu = validating(vdataloader, transformer.encoder, transformer.decoder, batch_size)
    transformer.train()
    scheduler.step(bleu)
    epoch_loss /= iter_size
    cur_lr = optimizer.param_groups[0]['lr']
    print("[Epoch %d] Loss: %f, lr: %f, val_bleu: %.3f, Time: %.3f" % (epoch, epoch_loss, cur_lr, bleu, (t2-t1)))
    if bleu > max_bleu:
        max_bleu = bleu
        torch.save(transformer.state_dict(), "./models/transformer.model") 

Start Training
0 200 400 600 800 1000 1200 1400 1600 1800 2000 2200 2400 2600 2800 3000 3200 3400 3600 3800 4000 ['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '看', '到', '了']
['好', '啊', '\n', '偶', '要', '睡', '覺', '了', '<EOS>']
['好', '啊', '\n', '你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了']
['好', '啊', '\n', '我', '們', '要', '去']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '<EOS>']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '看', '到', '了']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '看', '到', '一']
['好', '啊', '\n', '我', '們', '要', '去']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '看', '到', '了']
['好', '啊', '\n', '偶', '要', '睡', '覺', '了', '<EOS>']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '要', '去', '看', '到', '你', '們', '看', '到', '你']
['我', '也', '是', '不', '會', '不', '

['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '你', '們']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '你', '們']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '他', '們']
['好', '啊', '\n', '我', '們', '要', '去']
['好', '啊', '\n', '偶', '要', '睡', '覺', '了', '<EOS>']
['我', '們', '要', '去', '看', '到', '你', '們', '說', '你', '們', '說', '他', '們', '說', '他', '們', '說', '他']
['好', '啊', '\n', '偶', '要', '睡', '覺', '了', '<EOS>']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '看', '到', '一']
['好', '啊', '\n', '你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了']
['好', '啊', '\n', '偶', '要', '睡', '覺', '了', '<EOS>']
['我', '們', '要', '去', '看', '到', '你', '們', '說', '你', '們', '說', '他', '們', '說', '他', '們', '說', '他']
['我', '們', '要', '去', '看', '到', '你', '們', '說', '你', '們', '說', '你', '們', '說', '他', '們', '說', '他']
['好', '啊', '\n', '你', '傳', '送', '了',

['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '你', '們']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '他', '們']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '他', '們']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '他', '們']
['好', '啊', '\n', '偶', '要', '睡', '覺', '了', '<EOS>']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '你', '們']
['好', '啊', '\n', '偶', '要', '睡', '覺', '了', '<EOS>']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '看', '到', '一']
['我', '也', '是', '不', '會', '不', '會', '\n', '我', '們', '在', '去', '看', '到', '你', '們', '說', '你', '們']
['好', '啊', '\n', '偶', '們', '要', '去', '吃', '了', '<EOS>']
['我', '們', '要', '去', '看', '到', '你', '們', '說', '你', '們', '說', '他', '們', '說', '他', '們', '說', '他']
['好', '啊', '\n', '偶', '要', '睡', '覺', '了', '<EOS>']
['好', '啊', '\n',

0 200 400 600 800 1000 1200 1400 1600 1800 2000 2200 2400 2600 2800 3000 3200 3400 3600 3800 4000 ['我', '們', '也', '是', '要', '去', '看']
['好', '啊', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '啊', '\n', '我', '們', '要', '去', '吃', '飯', '了', '<EOS>']
['我', '們', '也', '是', '要', '去', '看', '到', '\n', '我', '們', '都', '是', '要', '去', '看']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['我', '們', '在', '哪', '裡', '面', '的', '\n', '我', '們', '要', '去', '看', '到', '了', '<EOS>']
['好', '啊', '\n', '我', '們', '要', '去', '吃', '飯', '了', '<EOS>']
['我', '們', '也', '是', '要', '去', '看']
['好', '啊', '\n', '偶', '們', '要', '去', '洗', '澡', '了', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '啊', '\n', '我', '們', '要', '去', '吃', '飯', '<EOS>']
['好', '啊', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', 

['我', '們', '在', '哪', '裡', '面', '的', '時', '候', '有', '沒', '有', '沒', '有', '沒', '有', '\n', '我', '們']
['我', '們', '在', '哪', '裡', '面', '的', '\n', '我', '們', '都', '是', '要', '去', '看', '到', '了', '<EOS>']
['我', '們', '在', '哪', '裡', '面', '的', '\n', '我', '們', '都', '是', '要', '去', '看', '到', '了', '<EOS>']
['好', '啊', '\n', '我', '們', '要', '去', '吃', '飯', '<EOS>']
['好', '啊', '\n', '偶', '們', '要', '去', '洗', '澡', '了', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '啊', '\n', '偶', '們', '要', '去', '吃', '飯', '了', '<EOS>']
['我', '覺', '得', '我', '們', '都', '是', '要', '去', '看', '到', '\n', '我', '們', '都', '是', '要', '去', '看']
['好', '啊', '\n', '我', '們', '要', '去', '吃', '飯', '<EOS>']
['好', '啊', '\n', '偶', '們', '要', '去', '吃', '飯', '了', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '啊', '\n', '我', '們', '

...

['對', '阿', '\n', '我', '們', '都', '吃', '了', '\n', '吃']
['哦', '哦', ' ', ' ', '那', '我', '們', '的', 'p', 'p', 't', 't', '就', '是', '要', '用', 's', 't', 'r']
['哦', '哦', '\n', '好', '阿', '\n', '我', '也', '覺', '得', '\n', '我', '是', '不', '是', '有', '點', '難', '過']
['嗯', '嗯', '<EOS>']
['好', '哇', '\n', '偶', '們', '要', '去', '吃', '飯', '了', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '哇', '\n', '偶', '們', '要', '去', '吃', '飯', '了', '<EOS>']
['哈']
['好', '啊', '\n', '那', '我', '們', '約', '十', '點', '半', '<EOS>']
['好', '～', '<EOS>']
['我', '們', '約', '1', '2', ':', '2', '0', '好', '了', '～', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '啊', '\n', '你', '要', '睡', '覺', '了', '嗎', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '吧', '\n', '偶', '們', '要', '去', '洗', '澡', '了', '<EOS>']
['我', '也', '不', '知', '道', 'X', 'D', '\n', '我', '也'

['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '這', '個', '嗎', '？']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '這', '個', '\n', '這']
['哈', '哈', '\n', '我', '也', '是', '\n', '你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '吧', '\n', '偶', '要', '吃', '飯', '了', '<EOS>']
['我', '覺', '得', '我', '們', '的', '是', '\n', '但', '是', '我', '們', '都', '不', '知', '道', '\n', '不', '過']
['不', '懂', '\n', '不', '能', '亂', '講', '話', '<EOS>']
['我', '覺', '得', '我', '們', '的', '是', '一', '個', '人', '\n', '不', '是', '要', '一', '個', '人', '\n', '不']
['我', '也', '不', '知', '道', '耶', '\n', '我', '們', '是', '不', '是', '要', '去', '的', '\n', '我', '們', '都']
['好', '～', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['對', '呢', '\n', '偶', '們', '要', '去', '吃', '飯', '了', '<EOS>']
['好', '吧', '\n', '晚', '安', '豬', '豬'

0 200 400 600 800 1000 1200 1400 1600 1800 2000 2200 2400 2600 2800 3000 3200 3400 3600 3800 4000 ['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '這', '個', '嗎', '？']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['我', '們', '是', '不', '是', '要', '去', '的', '人', '<EOS>']
['好', '啊', '\n', '那', '我', '們', '約', '十', '點', '半', '好', '了', '<EOS>']
['我', '覺', '得', '我', '們', '系', '上', '的', '是', '一', '個', '人', '的', '人', '\n', '我', '覺', '得', '我']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '這', '個', '嗎', '<EOS>']
['我', '也', '覺', '得', '我', '們', '很', '好', '吃', '的', '\n', '我', '們', '是', '不', '是', '很', '好', '吃']
['我', '也', '不', '知', '道', '=', ' ', '=', '<EOS>']
['好', '～', '<EOS>']
['好', '啊', '<EOS>']
['好', '啊', '<EOS>']
['好', '吧', '\n', '偶', '要', '睡', '覺', '了', '<EOS>']
['我', '們', '在', '吃', '飯', '\n

['對', '啊', '\n', '我', '們', '吃', '了', '一', '個', '\n', '我', '們', '吃', '了', '一', '個', '飯', '\n', '吃']
['我', '覺', '得', '我', '們', '的', '是', '\n', '我', '覺', '得', '我', '們', '的', '是', '這', '個', '\n', '我']
['我', '覺', '得', '我', '們', '是', '用', '的', '\n', '我', '覺', '得', '我', '們', '的', '是', '不', '是', '有']
['好', '啊', '\n', '我', '們', '約', '個', '時', '間', '\n', '我', '們', '約', '一', '點', '半', '<EOS>']
['好', '～', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '～', '<EOS>']
['我', '覺', '得', '我', '們', '的', '是', '一', '個', '人', '的', '人', '\n', '我', '覺', '得', '我', '們', '的']
['好', '啊', '<EOS>']
['好', '～', '<EOS>']
['我', '們', '都', '在', '台', '北', '\n', '我', '們', '都', '在', '台', '北', '\n', '我', '們', '約', '一', '點']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '啊', '\n', '你', '要', '睡', '覺', '了', '嗎', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '這', '個', '嗎

['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '這', '個', '嗎', '？']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '這', '個', '嗎', '？']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '這', '個', '嗎', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '這', '個', '\n', '你']
['好', '吧', '\n', '偶', '要', '吃', '飯', '了', '<EOS>']
['對', '阿', ' ', ' ', '我', '覺', '得', '我', '們', '都', '是', '一', '個', '人', '的', '\n', '但', '是', '我']
['不', '懂', '\n', '不', '能', '亂', '講', '話', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '這', '個', '是', '我', '的', '<EOS>']
['我', '也', '不', '知', '道', '耶', '\n', '我', '們', '是', '不', '是', '有', '一', '個', '人', '\n', '我', '們']
['好', '～', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['對', '呀', '\n', '偶', '們', '要', '去', '吃', '飯', '了', '<EOS>']
['好', '啊', '\n', '我', '們', '要', '睡',

In [36]:
print("Start Testing")
transformer.eval()
f = open("transformer_result.txt", "a")
score = validating(vdataloader, transformer.encoder, transformer.decoder, batch_size, f)
f.close()
print(score)

Start Testing
['好', '啊', '\n', '我', '也', '是', '想', '說', '\n', '你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片']
['對', '呢', '\n', '偶', '在', '想', '要', '去', '吃', '飯', '<EOS>']
['好', '啊', '<EOS>']
['好', '啊', '\n', '我', '們', '約', '十', '點', '半', '要', '去', '吃', '飯', '\n', '我', '們', '要', '去', '吃']
['好', '啊', '\n', '我', '們', '要', '去', '吃', '飯', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '啊', '\n', '我', '們', '要', '去', '吃', '飯', '\n', '我', '們', '要', '去', '吃', '飯', '\n', '我', '們']
['哈']
['好', '啊', '<EOS>']
['好', '啊', '<EOS>']
['好', '啊', '\n', '我', '在', '想', '要', '不', '要', '去', '吃', '飯', '\n', '不', '然', '要', '吃', '飯', '<EOS>']
['好', '吧', '\n', '偶', '們', '要', '去', '洗', '澡', '了', '<EOS>']
['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '啊', '\n', '偶', '們', '要', '去', '吃', '飯', '<EOS>'

['你', '傳', '送', '了', ' ', '1', ' ', '張', '相', '片', '。', '\n', '你', '傳', '送', '了', ' ', '1', ' ']
['好', '啊', '\n', '我', '們', '約', '十', '點', '半', '<EOS>']
['好', '啊', '\n', '偶', '要', '去', '洗', '澡', '了', '<EOS>']
['好', '吧', '\n', '偶', '們', '要', '去', '吃', '飯', '<EOS>']
['好', '啊', '\n', '偶', '們', '要', '去', '吃', '飯', '<EOS>']
['嗯', '嗯', '<EOS>']
['好', '啊', '\n', '偶', '要', '去', '洗', '澡', '了', '<EOS>']
['好', '啊', '\n', '偶', '們', '要', '去', '吃', '飯', '了', '<EOS>']
['不', '懂', '<EOS>']
['好', '吧', '\n', '偶', '要', '去', '洗', '澡', '了', '<EOS>']
['哈']
['我', '也', '不', '知', '道', 'X', 'D', '\n', '我', '也', '不', '知', '道', '\n', '你', '傳', '送', '了', ' ']
['好', '啊', '\n', '我', '們', '約', '十', '點', '半', '<EOS>']
['我', '也', '不', '知', '道', '=', ' ', '=', '<EOS>']
['好', '啊', '<EOS>']
['好', '吧', '\n', '丹', '丹', '要', '睡', '覺', '了', '嗎', '<EOS>']
['好', '啊', '<EOS>']
['好', '吧', '\n', '偶', '們', '要', '去', '吃', '飯', '<EOS>']
['好', '吧', '\n', '那', '我', '們', '約', '十', '點', '半', '要', '去', '吃', '飯', '<EOS>']
['你', '傳', '送', '了

In [54]:
BOS.size()

torch.Size([1, 1, 2])

In [17]:
transformer.load_state_dict(torch.load("models/transformer.model"))

In [31]:
query = "你傳送了一張相片"

q_seq = [word2ix["<BOS>"]] + [word2ix[word] for word in list(query)[:20]] + [word2ix["<EOS>"]]
pad_num = MAXLEN + 2 - len(q_seq)
q_seq += [word2ix["<PAD>"]] * pad_num
q_seq = np.array(q_seq).reshape(MAXLEN + 2, 1)
pos = np.arange(1, MAXLEN + 3).reshape(MAXLEN + 2, 1)
q_seq = np.concatenate((q_seq, pos), 1)
q_seq = torch.LongTensor(q_seq).unsqueeze(0).to(device)
BOS = torch.tensor([[[0,1]]]).to(device)
transformer.eval()
enc_output = transformer.encoder(q_seq)
for i in range(MAXLEN-1):
    if i == 0:
        dec_output = transformer.decoder(BOS, enc_output, q_seq)
    else:
        dec_input = torch.cat((BOS[:,:,0], dec_output.max(-1)[-1]), 1).unsqueeze(2)
        dec_pos = torch.arange(1, i+2).long().unsqueeze(0).repeat(1, 1).unsqueeze(2).to(device)
        dec_input = torch.cat((dec_input, dec_pos), 2)
        dec_output = transformer.decoder(dec_input, enc_output, q_seq)
pred = dec_output.max(2)[-1]
last_words = []
for wix in pred[0]:
    word = ix2word[wix.item()]
    last_words.append(word)
    if word == "<EOS>":
        last_words = last_words[:-1]
        break
    if len(set(last_words[-3:])) == 1 and len(last_words) >= 3:
        last_words = last_words[:-2]
        break
output = "".join(last_words).split("\n")[0]
print(output.replace("偶", "我"))

你傳送了 1 張相片。
